# Importing Eye- and pupil data

In [12]:
import sys, os
sys.path.insert(0,"..") # this is not needed if you have installed pypillometry
import pypillometry as pp
print(pp.__package_path__) # check that correct version is being used
import pandas as pd
import numpy as np
import pylab as plt

/Users/mmi041/Dropbox/work/projects/pupil/pypillometry/pypillometry


In [ ]:
pp.PupilData.from_file("../data/example_shortblinks.pd")

In [8]:
from pypillometry import EyeData, EyeDataDict

In [48]:
def __init__(self, 
                time: np.ndarray = None,
                left_x: np.ndarray = None,
                left_y: np.ndarray = None,
                left_pupil: np.ndarray = None,
                right_x: np.ndarray = None,
                right_y: np.ndarray = None,
                right_pupil: np.ndarray = None,
                event_onsets: np.ndarray = None,
                event_labels: np.ndarray = None,
                sampling_rate: float = None,
                name: str = None,
                fill_time_discontinuities: bool = True,
                keep_orig: bool = False):
    """
    Parameters
    ----------
    time: 
        timing array or `None`, in which case the time-array goes from [0,maxT]
        using `sampling_rate` (in ms)
    left_x, left_y, left_pupil:
        data from left eye (at least one of the eyes must be provided, both x and y)
        pupil is optional
    right_x, right_y, right_pupil:
        data from right eye (at least one of the eyes must be provided, both x and y)
        pupil is optional
    sampling_rate: float
        sampling-rate of the pupillary signal in Hz
    name: 
        name of the dataset or `None` (in which case a random string is selected)

    event_onsets: 
        time-onsets of any events in the data (in ms, matched in `time` vector)
    event_labels:
        for each event in `event_onsets`, provide a label
    keep_orig: bool
        keep a copy of the original dataset? If `True`, a copy of the object
        as initiated in the constructor is stored in member `original`
    fill_time_discontinuities: bool
        sometimes, when the eyetracker loses signal, no entry in the EDF is made; 
        when this option is True, such entries will be made and the signal set to 0 there
        (or do it later using `fill_time_discontinuities()`)
    """
    if time is None and sampling_rate is None:
        raise ValueError("Either `time` or `sampling_rate` must be provided")

    if (left_x is None or left_y is None) and (right_x is None or right_y is None):
        raise ValueError("At least one of the eye-traces must be provided (both x and y)")
    self.data=EyeDataDict(left_x=left_x, left_y=left_y, left_pupil=left_pupil,
                            right_x=right_x, right_y=right_y, right_pupil=right_pupil)
    ## name
    if name is None:
        self.name = self._random_id()
    else:
        self.name=name
    
    ## set time array and sampling rate
    if time is None:
        maxT=len(self.data)/sampling_rate*1000.
        self.tx=np.linspace(0,maxT, num=len(self.data))
    else:
        self.tx=np.array(time, dtype=float)

    self.missing=np.zeros_like(self.tx, dtype=bool)

    if sampling_rate is None:
        self.fs=np.round(1000./np.median(np.diff(self.tx)))
    else:
        self.fs=sampling_rate
        
    self.set_event_onsets(event_onsets, event_labels)

    ## start with empty history    
    self.history=[]            

    self.original=None
    if keep_orig: 
        self.original=self.copy()

    ## fill in time discontinuities
    if fill_time_discontinuities:
        self.fill_time_discontinuities()



pp.EyeData.__init__=__init__
pp.EyeData.set_onsets=set_onsets

In [53]:
fname_samples="../data/002_rlmw_samples.asc"
df=pd.read_table(fname_samples, index_col=False,
                  names=["time", "left_x", "left_y", "left_p",
                         "right_x", "right_y", "right_p"])

## Eyelink tracker puts "   ." when no data is available for x/y coordinates
left_x=df.left_x.values
left_x[left_x=="   ."] = np.nan
left_x = left_x.astype(float)

left_y=df.left_y.values
left_y[left_y=="   ."] = np.nan
left_y = left_y.astype(float)

right_x=df.right_x.values
right_x[right_x=="   ."] = np.nan
right_x = right_x.astype(float)

right_y=df.right_y.values
right_y[right_y=="   ."] = np.nan
right_y = right_y.astype(float)

fname_events="../data/002_rlmw_events.asc"
# read the whole file into variable `events` (list with one entry per line)
with open(fname_events) as f:
    events=f.readlines()

# keep only lines starting with "MSG"
events=[ev for ev in events if ev.startswith("MSG")]
experiment_start_index=np.where(["experiment_start" in ev for ev in events])[0][0]
events=events[experiment_start_index+1:]
df_ev=pd.DataFrame([ev.split() for ev in events])
df_ev=df_ev[[1,2]]
df_ev.columns=["time", "event"]
df_ev

/var/folders/28/_ftmv1_n41n48znrymwflm940000gp/T/ipykernel_41447/111312202.py:2: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df=pd.read_table(fname_samples, index_col=False,


,time,event
0,7269756,F_1
1,7270256,S_1_1_5
2,7271352,C_1_5_2
3,7271589,R_1_15
4,7273355,F_2
...,...,...
1315,8453467,R_300_18
1316,8455583,F_300
1317,8456083,P_300
1318,8456608,CP_300_1


In [55]:
d = pp.EyeData(time=df.time,
               left_x=left_x, left_y=left_y, left_pupil=df.left_p,
               event_onsets=df_ev.time, event_labels=df_ev.event,
               keep_orig=True)
d

EyeData(sarugopo, 37.6MiB):
 n               : 597301
 sampling_rate   : 500.0
 data            : ['left_x', 'left_y', 'left_pupil']
 nmiss           : 0
 perc_miss       : 0.0
 duration_minutes: 19.910033333333335
 start_min       : 121.07631666666666
 end_min         : 140.98631666666668
 glimpse         : EyeDataDict(vars=3,n=597301): 
  left_x (float64): 655.6, 655.7, 655.0, 654.5, 655.0...
  left_y (float64): 599.9, 598.9, 597.6, 597.8, 597.8...
  left_pupil (float64): 1121.0, 1122.0, 1124.0, 1126.0, 1126.0...

 History:
 *
 └ fill_time_discontinuities()